In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install openai==1.54.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [ ]:
from PIL import Image
from openai import OpenAI
import base64
import openai
import pillow_heif
import pandas as pd
import os

In [ ]:
# OpenAI 버전 확인
print(openai.__version__)

1.54.4


In [ ]:
os.environ["OPENAI_API_KEY"] = "" # -> OpenAI API key 입력
client = OpenAI()

In [ ]:
class Image2Text:
    # 생성자 함수
    def __init__(self, heic_path, output_dir):
        self.heic_path = heic_path                      # HEIC 이미지 경로
        self.output_dir = output_dir                    # JPG로 변환된 이미지를 저장할 경로
        self.jpg_path = None

    # HEIC 확장자를 JPG 확장자로 변환
    def convert_heic_to_jpg(self):
        heif_file = pillow_heif.open_heif(self.heic_path) # pillow_heif 라이브러리를 사용하여 HEIC 이미지를 열기
        image = Image.frombytes(                        # HEIC 이미지를 PIL 이미지로 변환
            heif_file.mode,
            heif_file.size,
            heif_file.data,
            "raw",
            heif_file.mode,
            heif_file.stride,
        )
        base_name = os.path.basename(self.heic_path)    # 파일 경로에서 파일 이름만 추출
        file_name, _ = os.path.splitext(base_name)      # 파일 이름과 확장자(heic)를 분리
        self.jpg_path = os.path.join(self.output_dir, f"{file_name}.jpg") # jpg로 변환된 이미지를 저장할 경로
        image.save(self.jpg_path, "JPEG")               # jpg로 변환된 이미지 파일을 output_dir 경로에 저장

    # 이미지를 JPG로 변환하고 base64로 인코딩하여 반환
    def convert_and_encode_image(self):
        self.convert_heic_to_jpg()
        with open(self.jpg_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 이미지에 대한 질문을 OpenAI API에 요청 후 응답 반환
    def ask_about_image(self, client):
        base64_image = self.convert_and_encode_image()
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # -> gpt-4o-mini 모델 사용
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": (
                                '이 사진에 나와있는 음식은 무엇인가요? '
                                '음식 이름에는 따옴표("")를 붙여서 답변해 주세요. '
                                '예시: 이 사진에 나와 있는 음식은 "닭갈비"입니다.'
                            ),
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            },
                        },
                    ],
                }
            ],
        )
        return response.choices[0].message.content

    # 응답 내용에서 음식 이름 추출
    def extract_food_name(self, response_content):
        sentences = response_content.split('.')
        for sentence in sentences:
            if '"' in sentence:
                food_name = sentence.split('"')[1]
                return food_name
        return None

    # 월 추출
    def extract_month(self):
        base_name = os.path.basename(self.heic_path)  # HEIC 파일명
        month = base_name.split('-')[1]  # '2024-04-20.HEIC'에서 '04' 추출
        return int(month)  # 월을 정수로 반환

    # 계절 추출
    def get_season(self, month):
        if 3 <= month <= 5:
            return '봄'
        elif 6 <= month <= 8:
            return '여름'
        elif 9 <= month <= 11:
            return '가을'
        else:
            return '겨울'

    # 응답 내용과 음식 이름, 월, 계절을 데이터프레임에 저장하고 기록
    def save_to_dataframe(self, food_name, month, season, csv_path):
        data = {
            # 'response_content': [response_content],
            'food_name': [food_name],
            'month': [month],
            'season': [season]
        }
        df = pd.DataFrame(data)
        if not os.path.isfile(csv_path):
            df.to_csv(csv_path, index=False)
        else:
            df.to_csv(csv_path, mode='a', header=False, index=False)


output_dir = "/content/drive/MyDrive/Colab Notebooks/A4/re_images_jpg"  # -> JPG로 변환된 이미지를 저장할 경로 입력
csv_path = "/content/drive/MyDrive/Colab Notebooks/A4/food_names.csv"  # -> CSV 파일 경로 입력



In [ ]:
# Define paths for the three HEIC images
heic_paths = [
    "/content/drive/MyDrive/Colab Notebooks/A4/re_images_heic/2024-04-20.HEIC",
    "/content/drive/MyDrive/Colab Notebooks/A4/re_images_heic/2024-07-14.HEIC",
    "/content/drive/MyDrive/Colab Notebooks/A4/re_images_heic/2024-09-08.HEIC",
    "/content/drive/MyDrive/Colab Notebooks/A4/re_images_heic/2024-09-21.HEIC",
    "/content/drive/MyDrive/Colab Notebooks/A4/re_images_heic/2024-10-26.HEIC"
]

# Loop through each HEIC image, convert, and process it
for heic_path in heic_paths:
    converter = Image2Text(heic_path, output_dir)
    response_content = converter.ask_about_image(client)
    print(response_content)

    # Extract the food name from the response content
    food_name = converter.extract_food_name(response_content)
    print(f"음식 이름: {food_name}")

    # Extract month and season
    month = converter.extract_month()
    season = converter.get_season(month)
    print(f"월: {month}, 계절: {season}")

    # Save the response content, food name, month, and season to a CSV file
    converter.save_to_dataframe(food_name, month, season, csv_path)


이 사진에 나와 있는 음식은 "사시미"입니다.
음식 이름: 사시미
월: 4, 계절: 봄
이 사진에 나와 있는 음식은 "전"입니다.
음식 이름: 전
월: 7, 계절: 여름
이 사진에 나와 있는 음식은 "샐러드"입니다.
음식 이름: 샐러드
월: 9, 계절: 가을
이 사진에 나와 있는 음식은 "치킨"입니다.
음식 이름: 치킨
월: 9, 계절: 가을
이 사진에 나와 있는 음식은 "피자"입니다.
음식 이름: 피자
월: 10, 계절: 가을


In [ ]:
df = pd.read_csv(csv_path)
df

,food_name,month,season
0,사시미,4,봄
1,전,7,여름
2,샐러드,9,가을
3,치킨,9,가을
4,피자,10,가을
